In [9]:
from hypersurface_tf import *
from generate_h import *
from biholoNN import *
import tensorflow as tf
import numpy as np

### Prepare the dataset:

In [10]:
z0, z1, z2, z3, z4 = sp.symbols('z0, z1, z2, z3, z4')
Z = [z0,z1,z2,z3,z4]
f = z0**5 + z1**5 + z2**5 + z3**5 + z4**5 + 0.5*z0*z1*z2*z3*z4
np.random.seed(123)
HS = Hypersurface(Z, f, 10000)
np.random.seed(124)
HS_test = Hypersurface(Z, f, 10000)

In [11]:
train_set = generate_dataset(HS)
test_set = generate_dataset(HS_test)

In [12]:
train_set = train_set.shuffle(500000).batch(1000)
test_set = test_set.shuffle(50000).batch(1000)

### Build the model:

In [21]:
class KahlerPotential(tf.keras.Model):

    def __init__(self):
        super(KahlerPotential, self).__init__()
        self.biholomorphic = Biholomorphic()
        self.layer1 = tf.keras.layers.Dense(100, activation=tf.square, use_bias=False)
        self.layer2 = tf.keras.layers.Dense(100, activation=tf.square, use_bias=False)
        #self.layer3 = tf.keras.layers.Dense(100, activation=tf.square, use_bias=False)
        #self.layer_4 = ComplexDense(50, 10, activation=tf.square)
        #self.layer_3 = ComplexDense(10, 15, activation=tf.square)
        #self.g = ComplexG(70)
        
    def call(self, inputs):
        x = self.biholomorphic(inputs)
        x = self.layer1(x)
        x = self.layer2(x)
        #x = self.layer3(x)
        #x = self.layer_4(x)
        #x = self.g(x)
        #x = tf.linalg.diag_part(tf.matmul(x, x, adjoint_b=True))
        #x = tf.math.log(x)
        x = tf.reduce_sum(x, 1)
        x = tf.math.log(x)
        return x

In [22]:
model = KahlerPotential()

In [23]:
@tf.function
def volume_form(x, Omega_Omegabar, mass, restriction):

    kahler_metric = complex_hessian(tf.math.real(model(x)), x)
    volume_form = tf.math.real(tf.linalg.det(tf.matmul(restriction, tf.matmul(kahler_metric, restriction, adjoint_b=True))))
    weights = mass / tf.reduce_sum(mass)
    factor = tf.reduce_sum(weights * volume_form / Omega_Omegabar)
    #factor = tf.constant(35.1774, dtype=tf.complex64)
    return volume_form / factor

In [24]:
#optimizer = tf.keras.optimizers.SGD(learning_rate=1e-1)
#learning_rate = 1
n_layers = 3
epochs = 6000

alpha = 0.001
beta1 = 0.9
beta2 = 0.999
eta = 1e-8

m = [0]*n_layers
v = [0]*n_layers
t = 0

for epoch in range(epochs):
    for step, (points, Omega_Omegabar, mass, restriction) in enumerate(train_set):
        with tf.GradientTape() as tape:
            
            omega = volume_form(points, Omega_Omegabar, mass, restriction)
            loss = weighted_MAPE(Omega_Omegabar, omega, mass)  
            grads = tape.gradient(loss, model.trainable_weights)
            #print(grads)

        t = t + 1
        i = 0
        for weight, grad in zip(model.trainable_weights, grads):
            
            m[i] = beta1 * m[i] + (1 - beta1) * grad
            v[i] = beta2 * v[i] + (1 - beta2) * grad**2
  
            alpha_t = alpha * np.sqrt(1 - beta2**t) / (1 - beta1**t)
            theta = alpha_t * m[i] / (tf.sqrt(v[i]) + eta)
            #print(theta)
            
            i = i + 1
            
            weight.assign_sub(theta)
    
        if step % 500 == 0:
            print("step %d: loss = %.4f" % (step, loss))
    
    test_loss = 0
    test_loss_old = 100
    
    for step, (points, Omega_Omegabar, mass, restriction) in enumerate(test_set):
        omega = volume_form(points, Omega_Omegabar, mass, restriction)
        test_loss += weighted_MAPE(Omega_Omegabar, omega, mass)
   
    test_loss = tf.math.real(test_loss).numpy()/(step+1)
    print("test_loss:", test_loss)
    
    # This part doesn't work right now
    if test_loss > test_loss_old:
        break
    test_loss_old = test_loss

step 0: loss = 0.6070
test_loss: 0.09932825088500977
step 0: loss = 0.0978
test_loss: 0.06426638126373291
step 0: loss = 0.0657
test_loss: 0.05879887104034424
step 0: loss = 0.0564
test_loss: 0.05617036342620849
step 0: loss = 0.0546
test_loss: 0.05415566444396973
step 0: loss = 0.0538
test_loss: 0.05303669452667236
step 0: loss = 0.0513
test_loss: 0.05133878231048584
step 0: loss = 0.0507
test_loss: 0.04886788845062256
step 0: loss = 0.0483
test_loss: 0.04734538555145264
step 0: loss = 0.0462
test_loss: 0.04464642524719238
step 0: loss = 0.0418
test_loss: 0.042682137489318844
step 0: loss = 0.0413
test_loss: 0.04032407760620117
step 0: loss = 0.0377
test_loss: 0.037876405715942384
step 0: loss = 0.0357
test_loss: 0.03602229118347168
step 0: loss = 0.0340
test_loss: 0.034733164310455325
step 0: loss = 0.0316
test_loss: 0.033231563568115234
step 0: loss = 0.0326
test_loss: 0.03134039878845215
step 0: loss = 0.0303
test_loss: 0.030648682117462158
step 0: loss = 0.0297
test_loss: 0.029255

step 0: loss = 0.0143
test_loss: 0.014604041576385498
step 0: loss = 0.0135
test_loss: 0.01464722990989685
step 0: loss = 0.0133
test_loss: 0.014336119890213012
step 0: loss = 0.0132
test_loss: 0.01455539584159851
step 0: loss = 0.0131
test_loss: 0.014279202222824097
step 0: loss = 0.0138
test_loss: 0.014589097499847412
step 0: loss = 0.0136
test_loss: 0.014656957387924194
step 0: loss = 0.0142
test_loss: 0.014251896142959596
step 0: loss = 0.0127
test_loss: 0.01470661759376526
step 0: loss = 0.0146
test_loss: 0.014461288452148438
step 0: loss = 0.0135
test_loss: 0.014452280998229981
step 0: loss = 0.0131
test_loss: 0.01420001745223999
step 0: loss = 0.0130
test_loss: 0.01477858543395996
step 0: loss = 0.0135
test_loss: 0.014307210445404053
step 0: loss = 0.0141
test_loss: 0.014119254350662231
step 0: loss = 0.0137
test_loss: 0.01390238881111145
step 0: loss = 0.0125
test_loss: 0.014655587673187255
step 0: loss = 0.0133
test_loss: 0.014095629453659058
step 0: loss = 0.0139
test_loss: 0

step 0: loss = 0.0118
test_loss: 0.012855548858642579
step 0: loss = 0.0122
test_loss: 0.012555787563323975
step 0: loss = 0.0113
test_loss: 0.012621641159057617
step 0: loss = 0.0114
test_loss: 0.013076064586639404
step 0: loss = 0.0127
test_loss: 0.013016525506973266
step 0: loss = 0.0118
test_loss: 0.013001410961151124
step 0: loss = 0.0132
test_loss: 0.012733206748962403
step 0: loss = 0.0122
test_loss: 0.012698205709457398
step 0: loss = 0.0113
test_loss: 0.012925715446472167
step 0: loss = 0.0120
test_loss: 0.012793762683868408
step 0: loss = 0.0117
test_loss: 0.012715648412704467
step 0: loss = 0.0120
test_loss: 0.0126580810546875
step 0: loss = 0.0115
test_loss: 0.012709435224533081
step 0: loss = 0.0117
test_loss: 0.012709603309631348
step 0: loss = 0.0112
test_loss: 0.01283471941947937
step 0: loss = 0.0125
test_loss: 0.013153269290924072
step 0: loss = 0.0129
test_loss: 0.012661217451095582
step 0: loss = 0.0115
test_loss: 0.012532508373260498
step 0: loss = 0.0115
test_loss

test_loss: 0.01232213020324707
step 0: loss = 0.0115
test_loss: 0.012053852081298827
step 0: loss = 0.0111
test_loss: 0.01229580283164978
step 0: loss = 0.0116
test_loss: 0.01229584813117981
step 0: loss = 0.0112
test_loss: 0.012347056865692138
step 0: loss = 0.0113
test_loss: 0.012108452320098877
step 0: loss = 0.0116
test_loss: 0.012210074663162231
step 0: loss = 0.0110
test_loss: 0.011870735883712768
step 0: loss = 0.0114
test_loss: 0.012217087745666504
step 0: loss = 0.0116
test_loss: 0.01262924551963806
step 0: loss = 0.0111
test_loss: 0.012287839651107788
step 0: loss = 0.0113
test_loss: 0.01221219778060913
step 0: loss = 0.0109
test_loss: 0.012246286869049073
step 0: loss = 0.0113
test_loss: 0.012103725671768189
step 0: loss = 0.0111
test_loss: 0.012257717847824097
step 0: loss = 0.0117
test_loss: 0.01205466628074646
step 0: loss = 0.0108
test_loss: 0.011964136362075805
step 0: loss = 0.0110
test_loss: 0.01202862024307251
step 0: loss = 0.0116
test_loss: 0.011995714902877808
ste

step 0: loss = 0.0110
test_loss: 0.012300717830657958
step 0: loss = 0.0111
test_loss: 0.011715128421783447
step 0: loss = 0.0107
test_loss: 0.011873233318328857
step 0: loss = 0.0109
test_loss: 0.011850458383560181
step 0: loss = 0.0110
test_loss: 0.011875067949295043
step 0: loss = 0.0107
test_loss: 0.011894387006759644
step 0: loss = 0.0109
test_loss: 0.011773816347122192
step 0: loss = 0.0108
test_loss: 0.01178048610687256
step 0: loss = 0.0112
test_loss: 0.011778953075408936
step 0: loss = 0.0103
test_loss: 0.011644768714904784
step 0: loss = 0.0113
test_loss: 0.01179140329360962
step 0: loss = 0.0114
test_loss: 0.0116320538520813
step 0: loss = 0.0109
test_loss: 0.011862061023712157
step 0: loss = 0.0110
test_loss: 0.01182376742362976
step 0: loss = 0.0109
test_loss: 0.01168299674987793
step 0: loss = 0.0103
test_loss: 0.011917111873626708
step 0: loss = 0.0112
test_loss: 0.011730700731277466
step 0: loss = 0.0105
test_loss: 0.011661229133605957
step 0: loss = 0.0112
test_loss: 0

...

step 0: loss = 0.0067
test_loss: 0.006985004544258117
step 0: loss = 0.0063
test_loss: 0.0069847780466079715
step 0: loss = 0.0060
test_loss: 0.007022766470909119
step 0: loss = 0.0061
test_loss: 0.007020527720451355
step 0: loss = 0.0064
test_loss: 0.007048904895782471
step 0: loss = 0.0064
test_loss: 0.007062922716140747
step 0: loss = 0.0064
test_loss: 0.007141086459159851
step 0: loss = 0.0068
test_loss: 0.007056018114089966
step 0: loss = 0.0060
test_loss: 0.007056000828742981
step 0: loss = 0.0064
test_loss: 0.007026289701461792
step 0: loss = 0.0066
test_loss: 0.006980472207069397
step 0: loss = 0.0065
test_loss: 0.007074512243270874
step 0: loss = 0.0065
test_loss: 0.007065949440002442
step 0: loss = 0.0065
test_loss: 0.007070410847663879
step 0: loss = 0.0063
test_loss: 0.0070761370658874514
step 0: loss = 0.0066
test_loss: 0.007127672433853149
step 0: loss = 0.0064
test_loss: 0.007128627300262451
step 0: loss = 0.0065
test_loss: 0.007027390003204345
step 0: loss = 0.0067
test

test_loss: 0.006966246962547302
step 0: loss = 0.0062
test_loss: 0.007041746377944946
step 0: loss = 0.0063
test_loss: 0.00695745050907135
step 0: loss = 0.0063
test_loss: 0.006996049284934997
step 0: loss = 0.0061
test_loss: 0.006998875141143799
step 0: loss = 0.0061
test_loss: 0.006998748183250427
step 0: loss = 0.0065
test_loss: 0.007036427259445191
step 0: loss = 0.0063
test_loss: 0.007074309587478638
step 0: loss = 0.0065
test_loss: 0.007033665776252747
step 0: loss = 0.0066
test_loss: 0.006999208331108093
step 0: loss = 0.0064
test_loss: 0.006966820955276489
step 0: loss = 0.0064
test_loss: 0.00700967013835907
step 0: loss = 0.0066
test_loss: 0.007016467452049255
step 0: loss = 0.0064
test_loss: 0.007168960571289062
step 0: loss = 0.0063
test_loss: 0.00699995756149292
step 0: loss = 0.0063
test_loss: 0.007131304740905762
step 0: loss = 0.0068
test_loss: 0.006984120011329651
step 0: loss = 0.0066
test_loss: 0.007054160237312317
step 0: loss = 0.0064
test_loss: 0.007028155922889709

step 0: loss = 0.0067
test_loss: 0.0070181721448898315
step 0: loss = 0.0068
test_loss: 0.007042770385742188
step 0: loss = 0.0065
test_loss: 0.0069933515787124634
step 0: loss = 0.0065
test_loss: 0.00703650951385498
step 0: loss = 0.0065
test_loss: 0.007051728963851929
step 0: loss = 0.0061
test_loss: 0.00699335515499115
step 0: loss = 0.0064
test_loss: 0.007127501368522644
step 0: loss = 0.0066
test_loss: 0.006936307549476623
step 0: loss = 0.0063
test_loss: 0.007038989067077637
step 0: loss = 0.0064
test_loss: 0.007152613401412964
step 0: loss = 0.0067
test_loss: 0.00696435272693634
step 0: loss = 0.0063
test_loss: 0.006983663439750671
step 0: loss = 0.0062
test_loss: 0.007034767270088196
step 0: loss = 0.0069
test_loss: 0.006943516731262207
step 0: loss = 0.0065
test_loss: 0.006917015910148621
step 0: loss = 0.0061
test_loss: 0.007020125985145569
step 0: loss = 0.0067
test_loss: 0.007040620446205139
step 0: loss = 0.0062
test_loss: 0.007029950022697449
step 0: loss = 0.0063
test_lo

step 0: loss = 0.0066
test_loss: 0.007069270014762878
step 0: loss = 0.0062
test_loss: 0.007030076384544373
step 0: loss = 0.0061
test_loss: 0.007072692513465881
step 0: loss = 0.0062
test_loss: 0.006982313990592956
step 0: loss = 0.0067
test_loss: 0.0070342087745666505
step 0: loss = 0.0063
test_loss: 0.00696378767490387
step 0: loss = 0.0062
test_loss: 0.007054601311683655
step 0: loss = 0.0063
test_loss: 0.007034944891929626
step 0: loss = 0.0065
test_loss: 0.007093643546104431
step 0: loss = 0.0063
test_loss: 0.006941607594490052
step 0: loss = 0.0065
test_loss: 0.006901085972785949
step 0: loss = 0.0066
test_loss: 0.007105556726455689
step 0: loss = 0.0063
test_loss: 0.006955658197402954
step 0: loss = 0.0064
test_loss: 0.00704873263835907
step 0: loss = 0.0062
test_loss: 0.007033311128616333
step 0: loss = 0.0064
test_loss: 0.007001749277114868
step 0: loss = 0.0064
test_loss: 0.007088199257850647
step 0: loss = 0.0066
test_loss: 0.007029955387115479
step 0: loss = 0.0064
test_lo

step 0: loss = 0.0059
test_loss: 0.007018402814865112
step 0: loss = 0.0063
test_loss: 0.006974183320999146
step 0: loss = 0.0064
test_loss: 0.007028198838233948
step 0: loss = 0.0063
test_loss: 0.007010753154754639
step 0: loss = 0.0064
test_loss: 0.006989938616752625
step 0: loss = 0.0064
test_loss: 0.006944034099578857
step 0: loss = 0.0063
test_loss: 0.0069385749101638795
step 0: loss = 0.0063
test_loss: 0.006959058046340942
step 0: loss = 0.0062
test_loss: 0.007022362947463989
step 0: loss = 0.0063
test_loss: 0.006914392709732055
step 0: loss = 0.0060
test_loss: 0.006998027563095093
step 0: loss = 0.0061
test_loss: 0.007120726108551026
step 0: loss = 0.0065
test_loss: 0.006988462209701538
step 0: loss = 0.0063
test_loss: 0.0069773197174072266
step 0: loss = 0.0063
test_loss: 0.006948070526123047
step 0: loss = 0.0065
test_loss: 0.0070305800437927245
step 0: loss = 0.0064
test_loss: 0.007030363082885742
step 0: loss = 0.0066
test_loss: 0.007033449411392212
step 0: loss = 0.0066
tes

### Some debugging tests:

In [ ]:
@tf.function
def volume_form(x, Omega_Omegabar, mass, restriction):

    kahler_metric = complex_hessian(tf.math.real(model(x)), x)
    volume_form = tf.linalg.det(tf.matmul(restriction, tf.matmul(kahler_metric, restriction, adjoint_b=True)))
    weights = mass / tf.reduce_sum(mass)
    factor = tf.reduce_sum(weights * volume_form / Omega_Omegabar)
    #factor = tf.constant(4.380538, dtype=tf.complex64)
    return  volume_form/factor
    #return factor
for step, (points, Omega_Omegabar, mass, restriction) in enumerate(dataset):
    omega = volume_form(points, Omega_Omegabar, mass, restriction)
    
    weights = mass / tf.reduce_sum(mass)
    print('omega', omega)
    print('OO',Omega_Omegabar)
    print(tf.cast(tf.abs(Omega_Omegabar -  omega), dtype=tf.complex64) / Omega_Omegabar)
   # print(mass/tf.reduce_sum(mass))

In [16]:
for weight in model.trainable_weights:
    print(weight)

<tf.Variable 'Variable:0' shape=(5, 15) dtype=complex64, numpy=
array([[ 2.2768168e+00+1.6352530e+00j,  5.8658237e+00-4.2718673e-01j,
         5.3873414e-01-1.7244281e-01j, -3.9534414e-01+7.2521999e-02j,
        -1.8923687e-02+1.1802919e-02j, -3.8347977e-01-2.2627316e-01j,
        -7.6662725e-01+8.0790210e-01j,  2.6793966e-01+6.7697294e-02j,
        -4.4106621e-01-1.5242220e-01j, -1.6198709e+00-5.7326740e-01j,
         2.8095651e+00+8.1460869e-01j, -1.1343197e+00-2.5883400e+00j,
        -4.6649609e+00+1.1073344e+00j,  1.7060937e-01-2.2714563e-02j,
         8.5827917e-01-7.9530555e-01j],
       [ 1.9180773e+00+4.4432992e-01j, -1.7982172e+00+1.0105304e+00j,
        -6.0589733e+00-8.5629034e-01j,  2.9838247e+00-3.7910095e-01j,
         2.8432586e+00+9.5829725e-01j, -4.8352519e-01-4.6606249e-01j,
        -1.7036586e+00-9.6777737e-01j, -3.2158126e-03+1.6477600e-03j,
         1.7850419e+00+6.1775154e-01j,  4.3784446e-01-8.8729739e-02j,
         2.8238079e-01+1.9228293e-01j, -2.9208440e-01+1.

In [21]:
class KahlerPotential(tf.keras.Model):

    def __init__(self):
        super(KahlerPotential, self).__init__()
        self.biholomorphic = Biholomorphic()
        self.layer1 = tf.keras.layers.Dense(100, activation=tf.square, use_bias=False)


        #self.layer_4 = ComplexDense(50, 10, activation=tf.square)
        #self.layer_3 = ComplexDense(10, 15, activation=tf.square)
        #self.g = ComplexG(70)
        
    def call(self, inputs):
        x = self.biholomorphic(inputs)
        x = self.layer1(x)
        #x = self.layer_2(x)

        #x = self.layer_4(x)
        #x = self.g(x)
        #x = tf.linalg.diag_part(tf.matmul(x, x, adjoint_b=True))
        #x = tf.math.log(x)
        x = tf.reduce_sum(x, 1)
        x = tf.math.log(x)
        return x

In [6]:
model = KahlerPotential()

In [13]:
for step, (points, Omega_Omegabar, mass, restriction) in enumerate(train_set):
    print(tf.shape(model(points)))
    if step == 0:
        a = model(points)

tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([1000], shape=(1,), dtype=int32)
tf.Tensor([

In [9]:
a = tf.transpose(a)
a = tf.reduce_sum(tf.abs(a), 1)

In [8]:
# TODO: Change Batchsize & Network structure

<tf.Tensor: shape=(1000, 1), dtype=float32, numpy=
array([[5.38316548e-01],
       [8.83447230e-02],
       [5.26222169e-01],
       [7.88766980e-01],
       [1.48804903e-01],
       [7.92228580e-01],
       [2.03350872e-01],
       [6.20962977e-01],
       [1.69604346e-01],
       [8.83976184e-03],
       [6.39118850e-02],
       [1.27156880e-02],
       [6.97958589e-01],
       [3.64552498e-01],
       [2.37538330e-02],
       [1.51279286e-01],
       [1.08250463e+00],
       [3.38911235e-01],
       [2.04938009e-01],
       [4.25134525e-02],
       [8.43953907e-01],
       [1.02830306e-03],
       [7.30025589e-01],
       [4.43385094e-01],
       [3.14193010e-01],
       [1.46317527e-01],
       [1.05443120e+00],
       [9.39430669e-02],
       [2.25267470e-01],
       [1.41357586e-01],
       [2.44012162e-01],
       [3.09310365e-03],
       [3.00194952e-03],
       [1.89879518e-02],
       [1.02200031e-01],
       [3.23994249e-01],
       [1.09909177e-01],
       [2.04617605e-01],

In [26]:
a**2

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 4], dtype=int32)>

In [26]:
a_real = tf.math.real(a)
a_imag = tf.math.imag(a)
a_new = tf.concat([a_real, a_imag], axis=1)

In [35]:
boolean_mask = tf.cast(a, dtype=tf.bool)              
no_zeros = tf.ragged.boolean_mask(a, boolean_mask)

In [9]:
tf.shape(a)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1000,   50], dtype=int32)>

In [30]:
a_real[0]

<tf.Tensor: shape=(25,), dtype=float32, numpy=
array([ 0.9038784 ,  0.84662837, -0.09585851, -0.663743  , -0.43639532,
        0.8466283 ,  0.95462674, -0.0439431 , -0.8095745 , -0.05158594,
       -0.09585851, -0.0439431 ,  0.02316958,  0.01710205,  0.14759119,
       -0.663743  , -0.8095745 ,  0.01710205,  0.7057893 , -0.09472027,
       -0.43639532, -0.05158594,  0.14759119, -0.09472027,  1.        ],
      dtype=float32)>

In [25]:
a = tf.constant([1, 2])